In [ ]:
import numpy as np
import pandas as pd
import random

In [ ]:
recipe_df = pd.read_excel('/content/drive/Shareddrives/2021 데이터청년캠퍼스/데이터베이스/recipe_final.xlsx')  #최종 레시피 데이터셋

FileNotFoundError: ignored

In [ ]:
recipe_df = recipe_df.drop(['Unnamed: 0'], axis=1)

NameError: ignored

In [ ]:
#채식 분류를 문자형에서 숫자로 바꿈
recipe_df['Vegetarian_class']=recipe_df['Vegetarian_class'].replace(['채소', '달걀', '우유(유제품)', '생선,조개', '가금류', '소,돼지'], [0,1,2,3,4,5])

In [ ]:
#비건 음식 인덱스 채우는 함수

def find_true_vege(vege,day,week):

  true_vege=recipe_df[recipe_df['Vegetarian_class']<=vege]
  true_vege_list=true_vege['RCP_SEQ'].values.tolist() 

  count=day*week

  recipe_li=[[[0 for _ in range(2)] for _ in range(3)] for _ in range(7)]

  week_list=[]
  avo_dupli=[]

  while count!=0:
    if count==0:
      break  
    week_idx=random.randint(0,6)
    while week_idx in week_list:
      week_idx=random.randint(0,6)
    week_list.append(week_idx)

    day_list=[]
    for j in range(day):
      day_idx=random.randint(0,2)
      while day_idx in day_list:
        day_idx=random.randint(0,2)
      day_list.append(day_idx)

      rec=np.random.choice(true_vege_list)
      while rec in avo_dupli: #레시피 선택 중복 x
        rec=np.random.choice(true_vege_list)
      avo_dupli.append(rec)

      recipe_li[week_idx][day_idx][0]=rec
      recipe_li[week_idx][day_idx][1]=1
      count-=1
  return recipe_li

In [ ]:
#나머지 식단 채우는 함수
def find_false_vege(recipe_li,vege):

  false_vege=recipe_df[recipe_df['Vegetarian_class']>vege]
  false_vege_list=false_vege['RCP_SEQ'].values.tolist()

  avo_dupli=[]

  for i in range(7):
    for j in range(3):
      if recipe_li[i][j][1]==0:
        rec=np.random.choice(false_vege_list) #레시피 선택 중복 x
        while rec in avo_dupli:
          rec=np.random.choice(false_vege_list)
        avo_dupli.append(rec)
        recipe_li[i][j][0]=rec
  return recipe_li

In [ ]:
"""
'추천메뉴 인덱스'와 사용자 맞춤 '채식 구분 변수'를 반환해주는 함수
"""

def recommend_recipe_index(vege,day,week): 
  user_recipe=find_true_vege(vege,day,week) #비건 음식 인덱스 채우는 함수
  final_user_recipe=find_false_vege(user_recipe,vege) # 남은 칸 일반식으로 채우기

  return final_user_recipe

In [ ]:
def show_recipe_details(final_user_recipe):
  df = pd.DataFrame()
  df_list = []
  isvege_list = []
  for day in final_user_recipe:
    for meal in day:
      idx = meal[0]
      is_vege = meal[1]
      df = df.append(recipe_df[recipe_df['RCP_SEQ']==idx])
      # df_list.append(recipe_df[recipe_df['RCP_SEQ']==idx])
      isvege_list.append(is_vege)
  # df = pd.DataFrame(df_list)  
  df['is_Vege'] = isvege_list
  df = df[['RCP_SEQ', 'RCP_NM','is_Vege', 'VEGE_CLASS_SEQ', 'RCP_WAY2', 'RCP_PAT2',
       'INFO_EMISSIONS', 'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT',
       'INFO_NA', 'HASH_TAG', 'ATT_FILE_NO_MAIN', 'ATT_FILE_NO_MK',
       'RCP_PARTS_DTLS', 'MANUAL01', 'MANUAL02', 'MANUAL03', 'MANUAL04',
       'MANUAL05', 'MANUAL06', 'MANUAL07', 'MANUAL08', 'MANUAL09', 'MANUAL10'
       ]]
  df = df.reset_index().drop(['index'],axis=1)
  return df

In [ ]:
def recipeRecommend_C(vege, day, week):

  emission = 1000 # 나올 수 없는 탄소배출량 값으로 초기값 설정

# 식단의 주간 탄소배출량이 27kg 이하가 될 때까지 식단 추천 반복
  while emission > 27:
    recipe_index = recommend_recipe_index(vege, day, week)
    recipe_all = show_recipe_details(recipe_index)
    emission = recipe_all['INFO_EMISSIONS'].sum(axis=0)/1000

  reco_recipe_idx = []
  reco_recipe_isvege = []

  for list2 in recipe_index:
    for list1 in list2:
      reco_recipe_idx.append(list1[0])
      reco_recipe_isvege.append(list1[1])

  return reco_recipe_idx, reco_recipe_isvege 